# Crypto Portfolio Backtesting System with Nautilus Trader

This notebook establishes a foundational system for backtesting and comparing different portfolio management strategies for cryptocurrencies using the `nautilus-trader` framework.

We will implement and compare two fundamental strategies:
1.  **Hodler Strategy**: Buys and holds BTC and ETH in equal dollar amounts at the start of the backtest.
2.  **Index Rebalance Strategy**: Buys BTC and ETH and periodically rebalances the portfolio to maintain a fixed 50/50 weight.

We will use 1-hour bar data for `BTC/USDT` and `ETH/USDT` downloaded from the official Binance public data repository.

In [ ]:
# !pip install nautilus_trader pyarrow pandas matplotlib

## 1. Imports and Setup

First, we import all the necessary components from `nautilus-trader` and other libraries for data handling and plotting. We'll also set up our logging and data directory.

In [2]:
import os
import shutil
import zipfile
from decimal import Decimal
from datetime import datetime, timedelta

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Core Nautilus Trader components
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.config import BacktestRunConfig, BacktestVenueConfig, BacktestDataConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.core.logging import LogLevel
from nautilus_trader.model.data import Bar, BarType, BarSpecification
from nautilus_trader.model.enums import AccountType, OrderSide, OrderType, TimeInForce
from nautilus_trader.model.events import OrderFilled
from nautilus_trader.model.identifiers import AssetId, InstrumentId, StrategyId, Venue
from nautilus_trader.model.instruments import Instrument
from nautilus_trader.model.objects import Money, Price, Quantity
from nautilus_trader.model.orders import MarketOrder
from nautilus_trader.model.position import Position
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.trading.strategy import Strategy

# Import the Binance adapter components
from nautilus_trader.adapters.binance.spot.loaders import BinanceSpotKlinesLoader
from nautilus_trader.adapters.binance.factories import BinanceDataFactory

# Create directories for our data
CATALOG_PATH = os.path.expanduser("~/backtest/data/catalog")
RAW_DATA_DIR = os.path.expanduser("~/backtest/data/raw")
if os.path.exists(RAW_DATA_DIR):
    shutil.rmtree(RAW_DATA_DIR)
if os.path.exists(CATALOG_PATH):
    shutil.rmtree(CATALOG_PATH)
os.makedirs(RAW_DATA_DIR, exist_ok=True)
os.makedirs(CATALOG_PATH, exist_ok=True)

print(f"Raw data directory: {RAW_DATA_DIR}")
print(f"Data catalog path: {CATALOG_PATH}")

ModuleNotFoundError: No module named 'nautilus_trader.core.logging'

## 2. Data Preparation

We will use the `BinanceSpotKlinesLoader` to process the CSV files from Binance. This loader is specifically designed to handle the format of their public data.

To make this notebook runnable without manual downloads, we'll first generate sample ZIP archives that mimic the structure of the official Binance data.

In [ ]:
# --- Create Sample Binance-formatted ZIP archives for demonstration ---
def create_sample_binance_zip(symbol, date_str, start_price, drift, volatility):
    """Creates a sample CSV and zips it to mimic Binance's public data format."""
    csv_filename = f"{symbol}-1h-{date_str}.csv"
    zip_filename = f"{symbol}-1h-{date_str}.zip"
    csv_filepath = os.path.join(RAW_DATA_DIR, csv_filename)
    zip_filepath = os.path.join(RAW_DATA_DIR, zip_filename)
    
    print(f"Creating sample file: {zip_filepath}")
    
    start = datetime.strptime(date_str, "%Y-%m")
    end = (start + pd.DateOffset(months=1)) - pd.Timedelta(hours=1)
    
    dates = pd.to_datetime(pd.date_range(start=start, end=end, freq='1H'))
    n_bars = len(dates)
    
    returns = np.random.normal(loc=drift, scale=volatility, size=n_bars)
    prices = start_price * (1 + returns).cumprod()
    
    df = pd.DataFrame()
    df['open_time'] = [int(d.timestamp() * 1000) for d in dates]
    df['open'] = prices
    df['high'] = prices * (1 + np.random.uniform(0, 0.01, n_bars))
    df['low'] = prices * (1 - np.random.uniform(0, 0.01, n_bars))
    df['close'] = prices * (1 + np.random.normal(0, 0.005, n_bars))
    df['volume'] = np.random.randint(100, 1000, n_bars)
    df['close_time'] = [int((d + timedelta(hours=1)).timestamp() * 1000 - 1) for d in dates]
    df['quote_asset_volume'] = df['volume'] * df['close']
    df['number_of_trades'] = np.random.randint(50, 500, n_bars)
    df['taker_buy_base_asset_volume'] = df['volume'] / 2
    df['taker_buy_quote_asset_volume'] = df['quote_asset_volume'] / 2
    df['ignore'] = 0
    
    df.to_csv(csv_filepath, header=False, index=False)
    
    # Create a zip file containing the CSV
    with zipfile.ZipFile(zip_filepath, 'w') as zf:
        zf.write(csv_filepath, arcname=csv_filename)
    
    os.remove(csv_filepath) # Clean up the raw CSV

create_sample_binance_zip("BTCUSDT", "2023-01", 20000, 0.0001, 0.02)
create_sample_binance_zip("ETHUSDT", "2023-01", 1500, 0.00015, 0.025)

# --- Ingest the data into the catalog using the Binance adapter ---
VENUE = Venue("BINANCE")
catalog = ParquetDataCatalog(CATALOG_PATH)

loader = BinanceSpotKlinesLoader(catalog=catalog, data_path=RAW_DATA_DIR)

loader.ingest(
    symbol="BTCUSDT",
    start="2023-01-01",
    end="2023-01-31",
)

loader.ingest(
    symbol="ETHUSDT",
    start="2023-01-01",
    end="2023-01-31",
)

print("\nData ingestion complete.")
catalog.list_bar_types()

## 3. Define Trading Instruments

We need to define the `Instrument` objects for the assets we want to trade. This includes information like the asset pair, venue, price precision, and quantity precision.

In [ ]:
# Configure a simulated venue with a simple percentage-based commission
venue_config = BacktestVenueConfig(
    name="BINANCE",
    oms_type="HEDGING", # Use HEDGING for simple portfolio management
    account_type=AccountType.CASH,
    base_currency=AssetId.from_str("USDT"),
    starting_balances=[Money(10_000, AssetId.from_str("USDT"))],
    # Apply a 0.1% taker fee to simulate exchange costs
    maker_fee_rate=Decimal("0.001"), 
    taker_fee_rate=Decimal("0.001"), 
)

instruments = BinanceDataFactory.create_spot_instruments(
    symbols=["BTC-USDT", "ETH-USDT"],
    venue=VENUE,
    price_precision=2,
    size_precision=5,
)

print("Defined Instruments:")
for inst in instruments:
    print(inst)

## 4. Implement Trading Strategies

Here we define the logic for our two strategies.

### Hodler Strategy

This is the simplest strategy. It allocates a portion of the starting capital to each asset, buys them on the first data tick (bar), and then holds them for the entire duration of the backtest.

In [ ]:
class Hodler(Strategy):
    """
    A simple buy-and-hold strategy. 
    Buys a fixed dollar amount of each subscribed instrument at the start.
    """
    def __init__(self, strategy_id: StrategyId, instruments: list[Instrument]):
        super().__init__(strategy_id)
        self.instruments = instruments
        self.invested_instruments = set()

    def on_start(self):
        """Called once at the start of the backtest."""
        self.subscribe_bars([BarType.from_instrument(inst, "1H") for inst in self.instruments])
        self.log.info(f"{self.strategy_id}: Subscribed to bars.")

    def on_bar(self, bar: Bar):
        """Called on each bar of data."""
        instrument = self.cache.instrument(bar.bar_type.instrument_id)
        if instrument.id in self.invested_instruments:
            return # Only invest once per instrument
        
        # Get total equity and calculate allocation per instrument
        account = self.portfolio.account(AccountType.CASH)
        total_equity = self.portfolio.equity(account)
        allocation_per_instrument = total_equity / len(self.instruments)

        self.log.info(
            f"{self.strategy_id}: Initial investment event for {instrument.id}. Total equity: {total_equity:.2f}"
        )

        # Calculate order quantity based on allocation and current price
        quantity = allocation_per_instrument.value / bar.close
        order_qty = instrument.make_quantity(quantity)
        
        self.log.info(
            f"{self.strategy_id}: Submitting market order for {order_qty} of {instrument.id}"
        )
        
        # Submit a market order to buy
        order = MarketOrder(
            strategy_id=self.id,
            instrument_id=instrument.id,
            quantity=order_qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC,
        )
        self.submit_order(order)
        self.invested_instruments.add(instrument.id)

    def on_order_filled(self, report: OrderFilled):
        """Log when an order has been filled."""
        self.log.info(
            f"{self.strategy_id}: FILLED {report.side} {report.filled_qty} of {report.instrument_id} at {report.avg_price}"
        )

    def on_stop(self):
        """Called once at the end of the backtest."""
        self.log.info(f"{self.strategy_id}: Backtest finished.")

### Index Rebalance Strategy

This strategy also starts by buying the assets, but it includes logic to periodically rebalance the portfolio to maintain a target weight for each asset. We'll set a rebalance interval (e.g., every 30 days) and on that schedule, the strategy will adjust the holdings by buying or selling.

In [ ]:
class IndexRebalance(Strategy):
    """
    A strategy that maintains a target weight for each instrument in the portfolio,
    rebalancing at a fixed interval.
    """
    def __init__(self, strategy_id: StrategyId, instruments: list[Instrument], rebalance_period_days: int = 30):
        super().__init__(strategy_id)
        self.instruments = instruments
        self.instrument_ids = [inst.id for inst in instruments]
        self.rebalance_period_ns = timedelta(days=rebalance_period_days).total_seconds() * 1e9
        
        # Target a 50/50 portfolio allocation
        self.target_weights = {inst.id: 1.0 / len(instruments) for inst in instruments}
        self.is_rebalancing = False
        self.initial_investment_done = False
        
    def on_start(self):
        """Set up subscriptions and schedule the rebalancing event."""
        bar_types = [BarType.from_instrument(inst, "1H") for inst in self.instruments]
        self.subscribe_bars(bar_types)
        self.log.info(f"{self.strategy_id}: Subscribed to bars.")
        
        # Schedule the first rebalance and all subsequent ones
        self.clock.schedule_interval(self.rebalance, self.rebalance_period_ns)

    def rebalance(self):
        """The core rebalancing logic."""
        if self.is_rebalancing:
            return
            
        self.log.info(f"{self.strategy_id}: === REBALANCING EVENT at {self.clock.timestamp_ns_to_datetime(self.clock.timestamp_ns)} ===")
        self.is_rebalancing = True
        
        account = self.portfolio.account(AccountType.CASH)
        total_equity = self.portfolio.equity(account)
        
        for instrument_id, target_weight in self.target_weights.items():
            target_value = total_equity * target_weight
            
            # Get current position and value
            position = self.portfolio.get_position(instrument_id)
            last_bar = self.get_last_bar(position.instrument_id)
            if last_bar is None:
                self.log.warn(f"No bar data yet for {instrument_id}, skipping rebalance.")
                self.is_rebalancing = False
                return
                
            current_price = last_bar.close
            current_value = position.quantity_as_decimal * current_price
            
            # Calculate the difference and the required trade quantity
            value_diff = target_value.value - current_value
            trade_qty_decimal = value_diff / current_price
            
            instrument = self.cache.instrument(instrument_id)
            if abs(trade_qty_decimal) < instrument.lot_size:
                self.log.info(f"{self.strategy_id}: No rebalance needed for {instrument_id}.")
                continue

            trade_qty = instrument.make_quantity(abs(trade_qty_decimal))
            side = OrderSide.BUY if trade_qty_decimal > 0 else OrderSide.SELL
            
            self.log.info(
                f"{self.strategy_id}: Rebalancing {instrument_id}. Target: ${target_value.value:,.2f}, Current: ${current_value:,.2f}. Will {side} {trade_qty}."
            )

            order = MarketOrder(
                strategy_id=self.id,
                instrument_id=instrument_id,
                quantity=trade_qty,
                side=side,
                time_in_force=TimeInForce.IOC,  # Use IOC for rebalancing orders
            )
            self.submit_order(order)
        
        self.is_rebalancing = False
        self.initial_investment_done = True

    def on_bar(self, bar: Bar):
        """Handle the initial investment on the first bar."""
        if not self.initial_investment_done:
            self.rebalance()

    def on_order_filled(self, report: OrderFilled):
        """Log filled orders."""
        self.log.info(
            f"{self.strategy_id}: FILLED {report.side} {report.filled_qty} of {report.instrument_id} at {report.avg_price}"
        )

    def on_stop(self):
        """Log end of backtest."""
        self.log.info(f"{self.strategy_id}: Backtest finished.")

## 5. Configure and Run the Backtest

Now we put everything together. We'll create instances of our strategies and configure the `BacktestNode` to run them simultaneously. This allows for a direct, side-by-side comparison.

In [ ]:
# Create unique IDs for our strategies
hodler_strategy_id = StrategyId("HODLER")
rebalance_strategy_id = StrategyId("REBALANCE")

config = BacktestRunConfig(
    data=BacktestDataConfig(catalog_path=CATALOG_PATH),
    venues=[venue_config],
    strategies=[
        Hodler(strategy_id=hodler_strategy_id, instruments=instruments),
        IndexRebalance(strategy_id=rebalance_strategy_id, instruments=instruments, rebalance_period_days=30),
    ],
)

# Instantiate and run the backtest node
node = BacktestNode(config)
results = node.run()

print("\nBacktest finished!")

## 6. Analyze and Compare Results

With the backtest complete, we can now analyze the performance of each strategy.

We'll extract the portfolio equity logs for both strategies and plot them on a chart to visualize their performance over time. We will also print the final equity for each.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')

# Extract portfolio logs for each strategy
hodler_log = results.trader.portfolio_log(strategy_id=hodler_strategy_id)
df_hodler = pd.DataFrame(hodler_log)
df_hodler['timestamp'] = pd.to_datetime(df_hodler['timestamp'], unit='ns')
df_hodler = df_hodler.set_index('timestamp')

rebalance_log = results.trader.portfolio_log(strategy_id=rebalance_strategy_id)
df_rebalance = pd.DataFrame(rebalance_log)
df_rebalance['timestamp'] = pd.to_datetime(df_rebalance['timestamp'], unit='ns')
df_rebalance = df_rebalance.set_index('timestamp')

# Plot the equity curves
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(df_hodler.index, df_hodler['equity'], label='Hodler Strategy', color='cyan')
ax.plot(df_rebalance.index, df_rebalance['equity'], label='Index Rebalance Strategy (30 days)', color='magenta')

ax.set_title('Strategy Performance Comparison (BTC/ETH)', fontsize=16)
ax.set_ylabel('Portfolio Equity (USDT)', fontsize=12)
ax.set_xlabel('Date', fontsize=12)
ax.legend(fontsize=12)

# Format y-axis to show dollar values
formatter = mticker.FormatStrFormatter('$%.0f')
ax.yaxis.set_major_formatter(formatter)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print final equity
final_equity_hodler = df_hodler['equity'].iloc[-1]
final_equity_rebalance = df_rebalance['equity'].iloc[-1]

print(f"Starting Equity: ${df_hodler['equity'].iloc[0]:,.2f}")
print(f"Final Equity (Hodler):      ${final_equity_hodler:,.2f}")
print(f"Final Equity (Rebalance):   ${final_equity_rebalance:,.2f}")

### Performance Statistics

Finally, let's generate and display detailed performance statistics for each strategy using the built-in reporting functionality.

In [ ]:
print("--- HODLER STRATEGY STATS ---")
results.trader.strategies[hodler_strategy_id].report.print()

print("\n--- INDEX REBALANCE STRATEGY STATS ---")
results.trader.strategies[rebalance_strategy_id].report.print()